In [6]:
import Bio
Bio.__version__

'1.78'

In [7]:
from Bio.SeqUtils import MeltingTemp as mt

The primer sequence is identical to the one used for the PCR Primer Inspector

In [8]:
print( mt.Tm_NN("ATGGCAGTTGAGAAGA", 
       nn_table=mt.DNA_NN4,
       Na=40,
       Tris=75.0,
       Mg=2.0, 
       dnac1=1000, 
       dnac2=0, 
       dNTPs=0.8, 
       saltcorr=7) )

57.794407437364214


The biopython module reports 57.79°C while the PCR Primer Inspector reported 54.4°C.
The difference is about 3.4 °C.

This is a greater difference that I expected with care taken to use the same thermodynamic data. 
I contacted Vladimir Cermak by email:

My email:

Hi, I have tried to replicate the
results from PCR Primer Inspector@molbiotools.com
with biopython using the same parameters and the same data
as far as I could tell. I still have a difference of about 3C.

I compiled the comparison in this notebook:

https://github.com/BjornFJohansson/tm/blob/master/tm.ipynb

Would you care to have a look at it? to see if there is something obviously wrong?
Is the code used by PCR Primer Inspector available for comparison?

All the best,
Björn Johansson

Response:

I tried to figure this out, but as I didn't see the python code I can only guess.

PCR Primer Inspector uses the parameters from [1], but changing them to [4] gives only 0.1°C difference.

What is probably the source of the discordant results is different understanding and implementation of this (from SantaLucia 1998):

    "For self-complementary oligonucleotide duplexes, the Tm is calculated from the predicted dH° and dS° and the total oligonucleotide strand concentration CT, by using the equation ...[3]... For non-selfcomplementary molecules, CT in Eq. 3 is replaced by CT/4 if the strands are in equal concentration or by (CA - CB/2) if the strands are at different concentrations, where CA and CB are the concentrations of the more concentrated and less concentrated strands, respectively."


That sounds like it should be CT for PCR conditions where C(primer)>>C(template) and CT/4 for an equimolar mix of complementary oligos. 
However, when checking the available code of Primer3, I found they use CT/4 for PCR conditions. 
I decided to be compliant with Primer3 (also Primer BLAST).

If I change the algorithm to CT instead of CT/4, it gives 57.0 for half the Tris concentration and 
57.9 for the whole Tris counted as monovalent cations.

So it may be that there is a mistake in Primer3 (and hence also in PCR Primer Inspector), 
which is a kind of "gold standard" in the field....

Best regards,
Vladimir Cermak


This means that the PCR Primer Inspector, Primer3 and other widely used tools implements the primer concentration
in a way not intended by the original author. The question is how to correctly emulate this using Biopython.
Fortunately the documentation explains this nicely:

[![](dnac1_2.png)](https://biopython.org/DIST/docs/api/Bio.SeqUtils.MeltingTemp-pysrc.html#Tm_NN)


If I instead assign dnac1 and dnac2 to half of the total primer concentration according to the biopyton docs,
the result become rather similar, especially if I also switch to the mt.DNA_NN3 table.

PCR Primer Inspector gave 54.4 while biopython below gave 0.8 C or 0.01 C higher tm.

In [9]:
from Bio.SeqUtils import MeltingTemp as mt
print( mt.Tm_NN("ATGGCAGTTGAGAAGA", 
       nn_table=mt.DNA_NN4,
       Na=40,
       Tris=75.0,
       Mg=2.0, 
       dnac1=1000/2, 
       dnac2=1000/2, 
       dNTPs=0.8, 
       saltcorr=7) )
print( mt.Tm_NN("ATGGCAGTTGAGAAGA", 
       nn_table=mt.DNA_NN3,
       Na=40,
       Tris=75.0,
       Mg=2.0, 
       dnac1=1000/2, 
       dnac2=1000/2, 
       dNTPs=0.8, 
       saltcorr=7) )

55.24658512819792
54.49960170791479


As per the email exchange above, PCR Primer Inspector follow the implementation of Primer3 with respect to the DNA concentration.  
[Primer3 - PCR primer design tool](https://github.com/primer3-org) is an old but widely usead and continuously updated program.

Primer3 is a c-program that I would like to avoid installing for now. Fortunately, there are several Primer3 instances available on the net.

A google search resulted in this primer3 instance: 

http://www.bioinformatics.nl/cgi-bin/primer3plus/primer3plus.cgi

I tried to emulate the settings used for PCR Primer Inspector as closely as possible. 
The Owczarzy 2008 salt correction is not available so Owczarzy 2004 was chosen.

![](primer3_settings2.png)

Monovalent and divalent cations, dNTPs and Annealing oligo concentrations were 
selected to reflect previous settings.

![](primer3_settings1.png)

The resulting Tm is 57C

![](primer3_result.png)

This is almost identical to the same settings in biopython:

In [10]:
print( mt.Tm_NN("ATGGCAGTTGAGAAGA", 
       nn_table=mt.DNA_NN3,
       Na=40,
       Tris=75.0,
       Mg=2.0, 
       dnac1=1000/2, 
       dnac2=1000/2, 
       dNTPs=0.8, 
       saltcorr=6) )

56.9591387892375


In [12]:
import pydna

In [13]:
pydna.__version__

'3.1.3.dev1+gca2d38a0dd'

In [11]:
from pydna.tm import tm_default

In [14]:
tm_default("ATGGCAGTTGAGAAGA")

52.64782631400061

The philosophy around the value for tm is more complex than I thought. The text below comes from the documentation for the Primer3 instance online 
[Primer3](http://www.bioinformatics.nl/cgi-bin/primer3plus/primer3plusHelp.cgi).

    Annealing Oligo Concentration

    The nanomolar concentration of annealing oligos in the PCR. Primer3 uses this argument to calculate oligo melting temperatures. The default (50nM) works well with the standard protocol used at the Whitehead/MIT Center for Genome Research--0.5 microliters of 20 micromolar concentration for each primer oligo in a 20 microliter reaction with 10 nanograms template, 0.025 units/microliter Taq polymerase in 0.1 mM each dNTP, 1.5mM MgCl2, 50mM KCl, 10mM Tris-HCL (pH 9.3) using 35 cycles with an annealing temperature of 56 degrees Celsius. This parameter corresponds to 'c' in Rychlik, Spencer and Rhoads' equation (ii) (Nucleic Acids Research, vol 18, num 21) where a suitable value (for a lower initial concentration of template) is "empirically determined". The value of this parameter is less than the actual concentration of oligos in the reaction because it is the concentration of annealing oligos, which in turn depends on the amount of template (including PCR product) in a given cycle. This concentration increases a great deal during a PCR; fortunately PCR seems quite robust for a variety of oligo melting temperatures.
    
 
I am not sure if this is still considered state of the art.